# Building a custom pipeline

The **deep**doctection analyzer is an example of a Document Layout Analysis pipeline. In this tutorial we'll show you the concepts so that you can build a pipeline youself.

In [1]:
from pathlib import Path
import deepdoctection as dd

The idea is not that difficult: There are models that fulfill a given task, there are pipeline components or pipeline backbones that invoke models and take care of pre- and post-processing results. There are also pipeline backbones that do not invoke models but only consolidate results. 

And there is the pipeline that puts everything together.

## Catalog and registries

You can get the essential information for pre-trained model from the `ModelCatalog`. 

In [ ]:
dd.print_model_infos(add_description=False,add_config=False,add_categories=False)

Let's select fasttext language detector. We need the categories that the model predicts and the model wrapper. `fasttext/lid.176.bin` is just an artefact. 

In [3]:
categories=dd.ModelCatalog.get_profile("fasttext/lid.176.bin").categories

In [4]:
dd.ModelCatalog.get_profile("fasttext/lid.176.bin").model_wrapper

'FasttextLangDetector'

We can download `lid.176.bin` with help of the `ModelDownloadManager`.

In [5]:
path_weights=dd.ModelDownloadManager.maybe_download_weights_and_configs("fasttext/lid.176.bin")

[1219 17:00.25 @fs.py:94]  INF  File lid.176.bin exists! Skip download.


## Model wrapper

We know from the `ModelCatalog` which wrapper we must use for the fasttext model.

In [6]:
fast_text = dd.FasttextLangDetector(path_weights, categories)

We are not done yet, because we still need to choose how to extract text. Let's simply stick to Tesseract and use the default english setting.

In [7]:
tess_ocr_config_path = dd.get_configs_dir_path() / "dd/conf_tesseract.yaml"  # This file will be in you .cache if you ran the analyzer before. 
# Otherwise make sure to copy the file from 'configs/conf_tesseract.yaml'

tesseract_ocr = dd.TesseractOcrDetector(tess_ocr_config_path.as_posix())

## Pipeline backbone

Similar to models we have a pipeline component registry. Having this starting point we can select the right backbone. Check the API documentation to see what the components are used for.

In [8]:
dd.pipeline_component_registry.get_all()

{'SubImageLayoutService': deepdoctection.pipe.cell.SubImageLayoutService,
 'ImageCroppingService': deepdoctection.pipe.common.ImageCroppingService,
 'MatchingService': deepdoctection.pipe.common.MatchingService,
 'PageParsingService': deepdoctection.pipe.common.PageParsingService,
 'LanguageDetectionService': deepdoctection.pipe.language.LanguageDetectionService,
 'ImageLayoutService': deepdoctection.pipe.layout.ImageLayoutService,
 'LMTokenClassifierService': deepdoctection.pipe.lm.LMTokenClassifierService,
 'LMSequenceClassifierService': deepdoctection.pipe.lm.LMSequenceClassifierService,
 'TableSegmentationRefinementService': deepdoctection.pipe.refine.TableSegmentationRefinementService,
 'TableSegmentationService': deepdoctection.pipe.segment.TableSegmentationService,
 'TextExtractionService': deepdoctection.pipe.text.TextExtractionService,
 'TextOrderService': deepdoctection.pipe.text.TextOrderService,
 'SimpleTransformService': deepdoctection.pipe.transform.SimpleTransformService

## Fasttext language detector

In [9]:
lang_detect_comp = dd.LanguageDetectionService(fast_text,text_detector=tesseract_ocr)

We can now build our very simple pipeline.

In [10]:
pipe_comp_list = [lang_detect_comp]
pipe = dd.DoctectionPipe(pipeline_component_list=pipe_comp_list)

In [13]:
image_path = Path.cwd() / "pics/samples/sample_3" 

![title](./pics/samples/sample_3/sample_3.png)

When running the pipe, we get the language in which the document was written. 

In [14]:
df = pipe.analyze(path=image_path)
df.reset_state()
dp = next(iter(df))
dp.language

|                                                                                                                                                                                                 |1/?[00:00<00:00,1291.75it/s]
[1219 17:03.05 @doctectionpipe.py:124]  INF  Processing sample_3.png
[1219 17:03.08 @context.py:131]  INF  LanguageDetectionService total: 2.6558 sec.


<Languages.german>

When getting the text, the response is somewhat disappointing.

In [15]:
dp.text

''

The reason for that is that `LanguageDetectionService` is not responsible for extracting text. It has an OCR model, but the output is only used as input feed to the language detector. The text however is not persisted. If we had added a `TextExtractionService` before `LanguageDetectionService` we could have omitted the OCR model in the `LanguageDetectionService`. 

## Tesseract OCR detector

In [16]:
tesseract_ocr = dd.TesseractOcrDetector(tess_ocr_config_path.as_posix(),["LANGUAGES=deu"])

In [17]:
tesseract_ocr.config

{'LANGUAGES': 'deu', 'LINES': False, 'psm': 11}

In [18]:
# setting run_time_ocr_language_selection=True will dynamically select the OCR model for text extraction based on 
# the predicted languages. This helps to get much improved OCR results, if you have documents with various languages.

text_comp = dd.TextExtractionService(tesseract_ocr, run_time_ocr_language_selection=True)
pipe_comp_list.append(text_comp)

In [19]:
df = pipe.analyze(path=image_path)
df.reset_state()
dp = next(iter(df))
dp.text

|                                                                                                                                                                                                 |1/?[00:00<00:00,1357.82it/s]
[1219 17:03.17 @doctectionpipe.py:124]  INF  Processing sample_3.png
[1219 17:03.20 @context.py:131]  INF  LanguageDetectionService total: 2.6773 sec.
[1219 17:03.22 @context.py:131]  INF  TextExtractionService total: 2.7287 sec.


''

This is something unexpected. Why don't we generate any text? We can clearly see that the `TextExtractionService` did its job.

In [20]:
word_sample = dp.words[0]
len(dp.words), word_sample.characters, word_sample.bbox, word_sample.reading_order 

(553, 'Anleihemärkte', [137.0, 158.0, 472.0, 195.0], None)

## Text ordering

The reason is, that we do not have inferred a reading order. If there is no reading order, there is no contiguous text. We treat text extraction as a character recognition problem only. If we want a reading order of predicted words, we need to do it ourself. So let's add the 'TextOrderService'.

In [21]:
order_comp = dd.TextOrderService(text_container=dd.LayoutType.word)
pipe_comp_list.append(order_comp)

At least, we got some text. The beginning sounds good. But once the text comes to the region where the second and third column also have text lines, the order service does not distinguish between columns. So we must identify columns. For that we use the layout analyzer.

In [22]:
df = pipe.analyze(path=image_path)
df.reset_state()
dp = next(iter(df))
dp.text

|                                                                                                                                                                                                 |1/?[00:00<00:00,1212.23it/s]
[1219 17:03.25 @doctectionpipe.py:124]  INF  Processing sample_3.png
[1219 17:03.28 @context.py:131]  INF  LanguageDetectionService total: 2.674 sec.
[1219 17:03.30 @context.py:131]  INF  TextExtractionService total: 2.8881 sec.
[1219 17:03.30 @context.py:131]  INF  TextOrderService total: 0.0146 sec.


' Anleihemärkte im Geschäftsjahr bis zum 31.12.2018 Schwieriges Marktumfeld Entwicklung der Leitzinsen in den USA und im Euroraum Die internationalen Anleihe- %p.a. märkte entwickelten sich im Geschäftsjahr 2018 unter- schiedlich und phasenweise sehr volatil. Dabei machte sich bei den Investoren zunehmend Nervosität breit, was in steigen- -1 u u u u u u u u u u u den Risikoprämien zum Aus- 12/08 12/09 12/10 12/11 12/12 12/13 12/14 12/15 12/16 12/17 12/18 druck kam. Grund hierfür waren BE Fed-Leitzins Quelle: Thomson Financial Datastream Turbulenzen auf der weltpoli- BE E28-Leitzins Stand: 31.12.2018 tischen Bühne, die die politi- schen Risiken erhöhten. Dazu zählten unter anderem populis- Zinswende nach Rekordtiefs Fiskalpolitik des US-Präsidenten tische Strömungen nicht nur bei Anleiherenditen? Donald Trump in Form von in den USA und Europa, auch Im Berichtszeitraum kam es an Steuererleichterungen und einer in den Emerging Markets, wie den Anleihemärkten - wenn Erhöhung der Staatsausg

## Layout service

It now depends on whether we use Tensorflow or PyTorch. We opt for PyTorch, just because the model runs on a CPU.
Make sure, that the model has been loaded to your .cache.

In [23]:
path_weights = dd.ModelCatalog.get_full_path_weights("layout/d2_model_0829999_layout_inf_only.pt")
path_configs = dd.ModelCatalog.get_full_path_configs("layout/d2_model_0829999_layout_inf_only.pt")
categories = dd.ModelCatalog.get_profile("layout/d2_model_0829999_layout_inf_only.pt").categories

categories

{'1': <LayoutType.text>,
 '2': <LayoutType.title>,
 '3': <LayoutType.list>,
 '4': <LayoutType.table>,
 '5': <LayoutType.figure>}

In [ ]:
layout_detector = dd.D2FrcnnDetector(path_configs,path_weights,categories,device="cpu")
layout_comp = dd.ImageLayoutService(layout_detector)

We need to make sure, that the `ImageLayoutService` has been invoked before `TextOrderService`.  

In [25]:
pipe_comp_list.insert(0,layout_comp)

In [ ]:
df = pipe.analyze(path=image_path)
df.reset_state()
dp = next(iter(df))
dp.layouts

In [27]:
dp.text, dp.layouts[0].text

('', '')

Now this looks weird again, doesn't it? However the reason is still quite simple. We now get an empty text string because once we have a non-empty `dp.layouts` the routine responsible for creating `dp.text` will try to get the text from the `Layout`'s. But we haven't run any method that maps a `word` to some `Layout` object. We need to specify this by applying a `MatchingService`. We will also have to slightly change the configuration of the  `TextOrderService`.

In [28]:
map_comp = dd.MatchingService(parent_categories=["text","title","list","table","figure"], child_categories=["word"],
                             matching_rule = 'ioa', threshold=0.6) # same setting as for the deepdoctection analyzer

order_comp = dd.TextOrderService(text_container=dd.LayoutType.word,
                                 floating_text_block_names=["text","title","list", "figure"],
                                 text_block_names=["text","title","list","table","figure"])

pipe_comp_list = [layout_comp, lang_detect_comp, text_comp, map_comp, order_comp]
pipe = dd.DoctectionPipe(pipeline_component_list=pipe_comp_list)

In [29]:
df = pipe.analyze(path=image_path)
df.reset_state()
dp = next(iter(df))
dp.text

|                                                                                                                                                                                                 |1/?[00:00<00:00,1237.99it/s]
[1219 17:03.41 @doctectionpipe.py:124]  INF  Processing sample_3.png
[1219 17:03.43 @context.py:131]  INF  ImageLayoutService total: 1.6445 sec.
[1219 17:03.45 @context.py:131]  INF  LanguageDetectionService total: 2.6846 sec.
[1219 17:03.48 @context.py:131]  INF  TextExtractionService total: 2.8263 sec.
[1219 17:03.48 @context.py:131]  INF  MatchingService total: 0.0041 sec.
[1219 17:03.48 @context.py:131]  INF  TextOrderService total: 0.0387 sec.


'\nAnleihemärkte im Geschäftsjahr bis zum 31.12.2018\nSchwieriges Marktumfeld Die internationalen Anleihe- märkte entwickelten sich im Geschäftsjahr 2018 unter- schiedlich und phasenweise sehr volatil. Dabei machte sich bei den Investoren zunehmend Nervosität breit, was in steigen- den Risikoprämien zum Aus- druck kam. Grund hierfür waren Turbulenzen auf der weltpoli- tischen Bühne, die die politi- schen Risiken erhöhten. Dazu zählten unter anderem populis- tische Strömungen nicht nur in den USA und Europa, auch in den Emerging Markets, wie zuletzt in Brasilien und Mexiko, wo Populisten in die Regie- rungen gewählt wurden. Der eskalierende Handelskonflikt zwischen den USA einerseits sowie Europa und China ande- rerseits tat sein übriges. Zudem ging Italien im Rahmen seiner Haushaltspolitik auf Konfronta- tionskurs zur Europäischen Uni- on (EU). Darüber hinaus verun- sicherte weiterhin der drohende Brexit die Marktteilnehmer, insbesondere dahingehend, ob der mögliche Austritt des Ver- e

Finally, we got it!